In [ ]:
# Check for GPU
!nvidia-smi

# Clone MMDetection repo
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

# Install dependencies
!pip install -qq openmim
!mim install -e .
!pip install -qq -r requirements/mminstall.txt
!pip install -qq mmengine mmcv>=2.0.0 mmdet

# Verify installation
!python tools/check_env.py

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!cp /content/drive/MyDrive/path/to/htc_r50_artifact.py configs/htc/
!cp /content/drive/MyDrive/path/to/htc_r50.pth checkpoints/
!cp -r /content/drive/MyDrive/path/to/data_artifacts .

In [ ]:
!python tools/train.py configs/htc/htc_r50_artifact.py --cfg-options device=cuda

after training

In [ ]:
!python tools/test.py configs/htc/htc_r50_artifact.py work_dirs/htc_r50_artifact/latest.pth --eval bbox


# htc... for cuda

In [ ]:
_base_ = './htc_r50_fpn_1x_coco.py'

# 1. Modify dataset classes
classes = ('artefact',)

# 2. Dataset paths for Colab
data_root = '/content/WatchdogAI/data_artifacts/'  # ✅ Adjusted for Colab's default path

train_dataloader = dict(
    batch_size=2,
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    batch_sampler=dict(type='AspectRatioBatchSampler'),
    dataset=dict(
        type='CocoDataset',
        data_root=data_root,
        ann_file='train/_annotations.coco.json',
        data_prefix=dict(img='train/'),
        metainfo=dict(classes=classes),
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='LoadAnnotations', with_bbox=True, with_mask=False),
            dict(type='Resize', scale=(1333, 800), keep_ratio=True),
            dict(type='RandomFlip', prob=0.5),
            dict(type='PackDetInputs')
        ]
    )
)

val_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type='CocoDataset',
        data_root=data_root,
        ann_file='valid/_annotations.coco.json',
        data_prefix=dict(img='valid/'),
        metainfo=dict(classes=classes),
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='Resize', scale=(1333, 800), keep_ratio=True),
            dict(type='LoadAnnotations', with_bbox=True, with_mask=False),
            dict(
                type='PackDetInputs',
                meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape', 'scale_factor'))
        ]
    )
)

test_dataloader = val_dataloader

val_evaluator = dict(
    type='CocoMetric',
    ann_file=data_root + 'valid/_annotations.coco.json',
    metric=['bbox']
)

test_evaluator = val_evaluator

# 3. Model definition with GPU-compatible NMS (removed `device='cpu'`)
model = dict(
    roi_head=dict(
        bbox_head=[
            dict(type='Shared2FCBBoxHead', num_classes=1),
            dict(type='Shared2FCBBoxHead', num_classes=1),
            dict(type='Shared2FCBBoxHead', num_classes=1)
        ]
    ),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[.0, .0, .0, .0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(type='SmoothL1Loss', beta=1.0 / 9.0, loss_weight=1.0),
        test_cfg=dict(
            nms_pre=2000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),  # ✅ No device='cpu'
            min_bbox_size=0
        )
    ),
    test_cfg=dict(
        rpn=dict(
            nms_pre=2000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),  # ✅ GPU default
            min_bbox_size=0
        ),
        rcnn=dict(
            score_thr=0.05,
            nms=dict(type='nms', iou_threshold=0.5),  # ✅ GPU default
            max_per_img=100
        )
    ),
    train_cfg=dict(
        rpn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.7,
                neg_iou_thr=0.3,
                min_pos_iou=0.3,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=256,
                pos_fraction=0.5,
                neg_pos_ub=-1,
                add_gt_as_proposals=False),
            allowed_border=0,
            pos_weight=-1,
            debug=False,
            nms=dict(type='nms', iou_threshold=0.7),
            nms_pre=2000,
            max_per_img=1000,
            min_bbox_size=0
        ),
        rpn_proposal=dict(
            nms_pre=2000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0
        ),
        rcnn=[
            dict(
                assigner=dict(
                    type='MaxIoUAssigner',
                    pos_iou_thr=0.5,
                    neg_iou_thr=0.5,
                    min_pos_iou=0.5,
                    ignore_iof_thr=-1),
                sampler=dict(
                    type='RandomSampler',
                    num=512,
                    pos_fraction=0.25,
                    neg_pos_ub=-1,
                    add_gt_as_proposals=True),
                mask_size=28,
                pos_weight=-1,
                debug=False
            ) for _ in range(3)
        ]
    )
)

# 4. Training loop
runner = dict(type='EpochBasedRunner', max_epochs=6)

# 5. Output location
work_dir = './work_dirs/htc_r50_artifact'

# 6. Pretrained weights
load_from = 'checkpoints/htc_r50.pth'  # ✅ Path inside Colab or use a full URL
